In [1]:
from starlette.testclient import TestClient
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass, field
from typing import List, Literal

set_pico_cls()
show(picolink)

In [2]:
db = database('trfc.db')

In [8]:
def get_unique(table, col, where='1=1', direction='ASC'):
    return [x[f'{col}'] for x in db.q(f"SELECT DISTINCT {table}.{col} FROM {table} WHERE {where} ORDER BY {col} {direction}")]

get_unique('results', 'generic_comp', 'game_type="Cup"')

['Anglo-Italian Cup',
 "Associate Members' Cup",
 'FA Cup',
 'FA Trophy',
 "Full Members' Cup",
 'League Cup',
 'Third Division Cup']

In [25]:
@dataclass
class Head2HeadAll:
    min_season: int
    max_season: int
    league_tiers: list[int]
    include_playoffs: bool
    cup_competitions: list['str']
    pens_as_draws: bool
    venues: list['str']
    min_meetings: int

In [29]:
def input_number(title: str, id:str, min:int, max:int, value:int): return Label(
    f'{title}',
    Input(
        type='number',
        id=f'{id}',
        min=min,
        max=max,
        value=value
    )
)

show(
    input_number('Minimum', 'min_season', 1921, 2024, 1921)
)

In [85]:
league_tiers = [
    '2: Championship',
    '3: League One',
    '4: League Two',
    '5: National League',
]

def input_checkboxes(items: list, name: str, val_length: int|None = None):
    return Fieldset(
        *[Label(
            Input(
                type='checkbox',
                name=name,
                value=i[:val_length],
                checked=True
            ), i) for i in items
        ]
    )

show(
    input_checkboxes(league_tiers, 'league_tiers', 1)
)

In [89]:
cup_competitions = get_unique('results', 'generic_comp', 'game_type="Cup"')

show(
    input_checkboxes(cup_competitions, 'cup_competitions')
)

In [95]:
venues = ['Home', 'Away', 'Neutral']

show(
    input_checkboxes(venues, 'venues', 1)
)

input_checkboxes(venues, 'venues', 1)

```html
<fieldset><label>    <input type="checkbox" name="venues" value="H" checked>
Home</label><label>    <input type="checkbox" name="venues" value="A" checked>
Away</label><label>    <input type="checkbox" name="venues" value="N" checked>
Neutral</label></fieldset>
```

In [90]:
def input_switch(label: str, id:str, value:bool, checked:bool): return Fieldset(
    Label(
        Input(
            type='checkbox',
            role='switch',
            id=f'{id}',
            value=value,
            checked=checked 
        ),
        f'{label}',
    )
)

show(input_switch('Include Playoffs', 'include_playoffs', True, True))

In [91]:
def input_radio(title: str, id:str, name:str, values:list, checked:list): return Fieldset(
    Legend(f'{title}'),
    *[Label(
        Input(
            type='radio', 
            name=name,
            value=value,
            checked=True if value == checked else False
        ),
        f"{value}"
    ) for value in values],
    id=f'{id}'
)

show(input_radio('Include Playoffs', 'include_playoffs', 'include_playoffs', ['Yes', 'No'], 'Yes'))

In [105]:
show(
    Form(
        H2('Select Seasons'),
        Grid(
            input_number('Minimum', 'min_season', 1921, 2024, 1921),
            input_number('Maximum', 'max_season', 1921, 2024, 2024)
        ),
        Hr(),
        H2('Leagues'),
        input_checkboxes(league_tiers, 'league_tiers'),
        input_switch('Include Playoffs', 'include_playoffs', True, True),
        Hr(),
        H2('Cup Competitions'),
        input_checkboxes(cup_competitions, 'cup_competitions'),
        input_switch('Treat one-off cup games decided by penalty shoot-out as draws', 'pens_as_draw', True, True),
        Hr(),
        H2('Venues'),
        input_checkboxes(venues, 'venues', 1),
        Hr(),
        input_number('Minimum Meetings', 'min_meetings', 1, 65, 10)
    )
)

Form(
    H2('Select Seasons'),
    Grid(
        input_number('Minimum', 'min_season', 1921, 2024, 1921),
        input_number('Maximum', 'max_season', 1921, 2024, 2024)
    ),
    Hr(),
    H2('Leagues'),
    input_checkboxes(league_tiers, 'league_tiers'),
    input_switch('Include Playoffs', 'include_playoffs', True, True),
    Hr(),
    H2('Cup Competitions'),
    input_checkboxes(cup_competitions, 'cup_competitions'),
    input_switch('Treat one-off cup games decided by penalty shoot-out as draws', 'pens_as_draw', True, True),
    Hr(),
    H2('Venues'),
    input_checkboxes(venues, 'venues', 1),
    Hr(),
    input_number('Minimum Meetings', 'min_meetings', 1, 65, 10)
)

```html
<form enctype="multipart/form-data">  <h2>Select Seasons</h2>
  <div class="grid">
    <div>
<label>Minimum        <input type="number" min="1921" max="2024" value="1921" id="min_season" name="min_season">
</label>    </div>
    <div>
<label>Maximum        <input type="number" min="1921" max="2024" value="2024" id="max_season" name="max_season">
</label>    </div>
  </div>
<hr>  <h2>Leagues</h2>
<fieldset><label>      <input type="checkbox" name="league_tiers" value="2: Championship" checked>
2: Championship</label><label>      <input type="checkbox" name="league_tiers" value="3: League One" checked>
3: League One</label><label>      <input type="checkbox" name="league_tiers" value="4: League Two" checked>
4: League Two</label><label>      <input type="checkbox" name="league_tiers" value="5: National League" checked>
5: National League</label></fieldset><fieldset><label>      <input type="checkbox" role="switch" value checked id="include_playoffs" name="include_playoffs">
Include Playoffs</label></fieldset><hr>  <h2>Cup Competitions</h2>
<fieldset><label>      <input type="checkbox" name="cup_competitions" value="Anglo-Italian Cup" checked>
Anglo-Italian Cup</label><label>      <input type="checkbox" name="cup_competitions" value="Associate Members' Cup" checked>
Associate Members&#x27; Cup</label><label>      <input type="checkbox" name="cup_competitions" value="FA Cup" checked>
FA Cup</label><label>      <input type="checkbox" name="cup_competitions" value="FA Trophy" checked>
FA Trophy</label><label>      <input type="checkbox" name="cup_competitions" value="Full Members' Cup" checked>
Full Members&#x27; Cup</label><label>      <input type="checkbox" name="cup_competitions" value="League Cup" checked>
League Cup</label><label>      <input type="checkbox" name="cup_competitions" value="Third Division Cup" checked>
Third Division Cup</label></fieldset><fieldset><label>      <input type="checkbox" role="switch" value checked id="pens_as_draw" name="pens_as_draw">
Treat one-off cup games decided by penalty shoot-out as draws</label></fieldset><hr>  <h2>Venues</h2>
<fieldset><label>      <input type="checkbox" name="venues" value="H" checked>
Home</label><label>      <input type="checkbox" name="venues" value="A" checked>
Away</label><label>      <input type="checkbox" name="venues" value="N" checked>
Neutral</label></fieldset><hr><label>Minimum Meetings    <input type="number" min="1" max="65" value="10" id="min_meetings" name="min_meetings">
</label></form>
```

In [13]:
seasons = get_unique('results', 'season', direction='DESC')

ssn_dropdown = Details(
    Summary('Select seasons'),
    Ul(
        Li(Label(Input(type='checkbox', name=ssn[:4], checked=True if ssn==max(seasons) else False), ssn) for ssn in seasons)
    ),
    cls='dropdown'
)

show(ssn_dropdown)